# Examples of WaporAPI module functions

## 0. Change working directory to "wapor/src"

In [1]:
import os
import glob

p_from = os.getcwd()
print('p_from')
# for file in os.listdir(p_from):
#     print('  ', file)

p_root = os.path.join(os.getcwd(), '../')
print('p_root')
# for file in os.listdir(p_root):
#     print('  ', file)

p_wapor = os.path.join(p_root, 'src')
p_tests = os.path.join(p_root, 'tests')

os.chdir(p_wapor)
print('Working directory changed from "{f}" to "{t}"'.format(f=p_from, t=p_wapor))
# for file in os.listdir(os.getcwd()):
#     print('  ', file)

p_from
p_root
Working directory changed from "/notebooks/notebooks" to "/notebooks/notebooks/../src"


## 1. First import the module
Init module with API input.

In [ ]:
# from wapor.WaporAPI import __wapor_API_class
# API = __wapor_API_class()
# API.Token=input('Insert WAPOR API Token: ')

import wapor as WaPOR

with open('wapor/token.yml', 'r') as fp:
    token = fp.readline()
WaPOR.API.setAPIToken(token)

In [ ]:
AccToken = WaPOR.API.token['Access']
RefToken = WaPOR.API.token['Refresh']

print('\nAccess  Token:', AccToken)
print('\nRefresh Token:', RefToken)

Check Workspaces list

In [ ]:
workspaces = WaPOR.API.getWorkspaces()

for workspace in workspaces:
    print(workspace['code'])

## 2. Get Catalog

In [ ]:
version = 2
level = 2
catalog = WaPOR.API.getCatalog(version=version, level=level)

catalog

## 3. Get information of a cube

In [ ]:
cube_code='L1_PCP_M'
cube_info=WaPOR.API.getCubeInfo(cube_code)

cube_info

Some useful information about dataset can be found in cube_info

In [ ]:
print('Description: {0} \n\nmultiplier: {1} \n\nunit: {2} \n'.format(
    cube_info['description'],
    cube_info['measure']['multiplier'],
    cube_info['measure']['unit']))

## 4. Get list of locations

In [ ]:
WaPOR_version = 2
WaPOR_level = 2
df_locations=WaPOR.API.getLocations(version=WaPOR_version, level=WaPOR_level)

df_locations

## 5. Get list of available data

In [ ]:
df_avail=WaPOR.API.getAvailData(cube_code,
                                time_range='2009-01-01,2012-12-31')

df_avail

## 6. Get download url of a raster
To get download url, wapor API Token is needed

In [ ]:
print('WaPOR_{v}.L{l}'.format(v=WaPOR.API.version, l=WaPOR.API.level))

WaPOR.API.getRasterUrl(cube_code,
                       'L1_PCP_0901M')

## 7. Get download url of a cropped raster
To request a cropped raster, raster_id and time_code of the raster is needed

In [ ]:
print('WaPOR_{v}.L{l}'.format(v=WaPOR.API.version, l=WaPOR.API.level))

bbox = [37.95883206252312, 7.89534, 43.32093, 12.3873979377346]
time_code = '[2009-01-01,2009-02-01)'
rasterId = 'L1_PCP_0901M'
download_url=WaPOR.API.getCropRasterURL(bbox,
                                        cube_code,
                                        time_code,
                                        rasterId)

download_url

## 8. Get Area time-series
A shapefile can be used to define the Region of Interest

### 8.1 Check Shapefile folder

In [ ]:
Dir_Shapefile = os.path.join(p_tests, 'data', 'Shapefile')
chk_files=glob.glob(os.path.join(Dir_Shapefile, '*.shp'))

for file in chk_files:
    print(file)

### 8.2 Get shapefile area timeseries data

In [ ]:
print('WaPOR_{v}.L{l}'.format(v=WaPOR.API.version, l=WaPOR.API.level))

shp_fh=os.path.join(Dir_Shapefile, "Awash_HydroSHED.shp")
ts_area=WaPOR.API.getAreaTimeseries(shp_fh,
                                    'L1_PCP_M',
                                    time_range="2009-01-01,2018-12-31")

ts_area

## 9. Get Point time-series

In [ ]:
print('WaPOR_{v}.L{l}'.format(v=WaPOR.API.version, l=WaPOR.API.level))

Point=[37.95883206252312, 7.89534] #longitude and latitude
ts_point=WaPOR.API.getPixelTimeseries(Point,
                                      'L1_PCP_M',
                                      time_range="2009-01-01,2018-12-31")

ts_point

## 10. Bulk-downloading

### 10.1 Check Download folder

In [ ]:
import requests
Dir_Download = os.path.join(p_tests, 'data', 'Notebooks')

if not os.path.exists(Dir_Download):
    os.makedirs(Dir_Download)

chk_files=glob.glob(os.path.join(Dir_Download, '*.tif'))
for file in chk_files:
    print(file)

### 10.2 Start to download

In [ ]:
print('WaPOR_{v}.L{l}'.format(v=WaPOR.API.version, l=WaPOR.API.level))

for index,row in df_avail.iterrows():   
    download_url=WaPOR.API.getCropRasterURL(bbox,
                                            cube_code,
                                            row['time_code'],
                                            row['raster_id'])
    
    download_file=os.path.join(Dir_Download,
                               '{0}.tif'.format(row['raster_id']))
    
    resp=requests.get(download_url) 
    open(download_file, 'wb').write(resp.content)
    print(index, download_file)

### 10.3 Check downloaded files

In [ ]:
chk_files=glob.glob(os.path.join(Dir_Download, '*.tif'))
for file in chk_files:
    print(file)